In [40]:
import numpy as np
import pandas as pd
from gensim import models,corpora
from gensim import similarities
import pyLDAvis.gensim
import warnings
import re
import spacy

In [41]:
pd.set_option('max_colwidth',400)
warnings.filterwarnings('ignore')
pyLDAvis.enable_notebook()
PATH ='../data/'

In [46]:
nlp = spacy.load('en')
def clean_up_spacy(text):
    text_out = set()
    doc= nlp(text)
    for token in doc:         
        if  len(token)<15 and token.is_punct is False :
            if token.text != '':
                text_out.add(token.text)
    text_out = list(text_out)
    return text_out

# Loading the pretrained LDA model

In [12]:
Lda = models.LdaMulticore
lda_final =Lda.load(PATH+'lda_final')
dictionary = corpora.Dictionary.load(PATH+'dictionary')
doc_term_matrix = corpora.MmCorpus(PATH+'doc_term_matrix.mm')

# Feature Engineering

## Document-topic matrix : Used for automated Document tagging

### Tagging training data

In [13]:
doc2topic =lda_final.get_document_topics(doc_term_matrix,minimum_probability=0)
doc2topic = pd.DataFrame(list(doc2topic))
num_topics = lda_final.num_topics
doc2topic.columns = ['Topic'+str(i+1) for i in range(num_topics)]
for i in range(len(doc2topic.columns)):
    doc2topic.iloc[:,i]=doc2topic.iloc[:,i].apply(lambda x: x[1])
doc2topic['Automated_topic_id'] =doc2topic.apply(lambda x: np.argmax(x),axis=1)
doc2topic.head()

,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Automated_topic_id
0,0.006410,0.006410,0.006410,0.006410,0.006410,0.006410,0.736178,0.006410,0.006410,0.199718,0.006410,0.006411,Topic7
1,0.003968,0.003968,0.003968,0.003968,0.003968,0.003968,0.003968,0.003968,0.003969,0.956348,0.003968,0.003968,Topic10
2,0.005208,0.005208,0.005208,0.005208,0.005208,0.005208,0.005209,0.005208,0.005208,0.005209,0.005208,0.942708,Topic12
3,0.002604,0.042106,0.002604,0.002604,0.002604,0.002604,0.002604,0.002604,0.200676,0.245217,0.002604,0.491168,Topic12
4,0.004902,0.412774,0.004902,0.004902,0.004902,0.004902,0.004902,0.004902,0.004902,0.004902,0.205122,0.337986,Topic2


In [42]:
data = pd.read_csv(PATH+'input_data.csv',index_col='Unnamed: 0')
data_final =pd.concat([data,doc2topic],axis=1)
data_final.head(1)

,id,Skills,Job title,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Automated_topic_id
0,1.0,"['TECHNICAL SKILLS\xa0', <br/>, '\xa0', <br/>, '• R • Tableau • Machine Learning\xa0', <br/>, '• D3.js • SQL, PostgreSQL, pgadmin 4 • JavaScript\xa0', <br/>, '• Python • HTML/CSS • ...",Data Scientist Intern,0.00641,0.00641,0.00641,0.00641,0.00641,0.00641,0.736178,0.00641,0.00641,0.199718,0.00641,0.006411,Topic7


### Tagging unseen data

In [51]:
text = "pandas, scikit, numpy, matplotlib, regression, classification, python, r, sql, tableau, hadoop"
bow = dictionary.doc2bow(clean_up_spacy(text))

# lets see how our model fares with the given data. The above skills matches that of a data scientist. 


Topic_probability =pd.DataFrame(lda_final[bow])
Topic_probability.columns = ['Topic','Probability']
Topic_probability['Topic'] = Topic_probability['Topic'].apply(lambda x: 'Topic'+str(x+1))

Topic_probability.sort_values(by='Probability',ascending =False)

,Topic,Probability
6,Topic7,0.923611
11,Topic12,0.006945
8,Topic9,0.006945
9,Topic10,0.006945
1,Topic2,0.006944
4,Topic5,0.006944
0,Topic1,0.006944
2,Topic3,0.006944
5,Topic6,0.006944
7,Topic8,0.006944


In [43]:
num_topics =12
vis = pyLDAvis.gensim.prepare(lda_final, doc_term_matrix, dictionary,sort_topics=False)
pyLDAvis.save_html(vis,f'pyLDAvis_{num_topics}.html')
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0       7.236537        1       1 -0.028248 -0.179300
1       3.747244        1       2  0.012753 -0.027564
2       2.335483        1       3  0.122807  0.011622
3       2.675107        1       4  0.151100 -0.037611
4       3.369806        1       5  0.074784  0.057546
5       2.190405        1       6  0.132711  0.022145
6      20.327776        1       7 -0.204531  0.141215
7       3.049178        1       8  0.158089  0.021893
8      17.502689        1       9 -0.230710 -0.043406
9      14.408374        1      10 -0.171886 -0.107890
10      2.150024        1      11  0.161517  0.042300
11     21.007380        1      12 -0.178386  0.099050, topic_info=     Category         Freq          Term        Total  loglift  logprob
term                                                                   
81    Default  1529.000000           sas  1529.000000  30.0000  30.0000
99    Default   701.000000        pandas   701.000000  29.0000  29.0000
21    Default  1021.000000    javascript  1021.000000  28.0000  28.0000
75    Default  1057.000000          hive  1057.000000  27.0000  27.0000
2     Default  1114.000000          html  1114.000000  26.0000  26.0000
0     Default   657.000000           css   657.000000  25.0000  25.0000
150   Default   567.000000         numpy   567.000000  24.0000  24.0000
100   Default   458.000000        scikit   458.000000  23.0000  23.0000
204   Default   646.000000           pig   646.000000  22.0000  22.0000
7     Default  3288.000000             r  3288.000000  21.0000  21.0000
6     Default  3499.000000        python  3499.000000  20.0000  20.0000
27    Default  1091.000000         excel  1091.000000  19.0000  19.0000
31    Default  1462.000000        matlab  1462.000000  18.0000  18.0000
20    Default  1852.000000          java  1852.000000  17.0000  17.0000
139   Default   396.000000           php   396.000000  16.0000  16.0000
39    Default   357.000000         hbase   357.000000  15.0000  15.0000
252   Default   300.000000    matplotlib   300.000000  14.0000  14.0000
82    Default   954.000000         spark   954.000000  13.0000  13.0000
47    Default   515.000000    powerpoint   515.000000  12.0000  12.0000
898   Default   286.000000         sqoop   286.000000  11.0000  11.0000
13    Default  1582.000000             c  1582.000000  10.0000  10.0000
259   Default   359.000000         scipy   359.000000   9.0000   9.0000
195   Default   683.000000          spss   683.000000   8.0000   8.0000
253   Default   302.000000         learn   302.000000   7.0000   7.0000
216   Default   513.000000           xml   513.000000   6.0000   6.0000
78    Default   337.000000    regression   337.000000   5.0000   5.0000
269   Default   257.000000        jquery   257.000000   4.0000   4.0000
38    Default  1346.000000        hadoop  1346.000000   3.0000   3.0000
93    Default   369.000000           svm   369.000000   2.0000   2.0000
107   Default   357.000000    clustering   357.000000   1.0000   1.0000
...       ...          ...           ...          ...      ...      ...
81    Topic12  1052.151855           sas  1529.397461   1.1863  -3.4139
106   Topic12   306.742340        access   418.589203   1.2494  -4.6465
59    Topic12   307.866760      computer   438.340210   1.2070  -4.6429
742   Topic12    43.179939         maple    53.708900   1.3421  -6.6072
567   Topic12    34.140465  spss_modeler    42.242458   1.3474  -6.8421
31    Topic12   849.904724        matlab  1462.076904   1.0178  -3.6274
170   Topic12   129.096527         latex   185.167892   1.1996  -5.5120
7     Topic12  1539.529663             r  3288.466309   0.8014  -3.0333
1445  Topic12    95.513863   mathematica   136.292557   1.2048  -5.8133
8     Topic12  1367.497803           sql  3208.843506   0.7074  -3.1518
6     Topic12  1385.707520        python  3499.974609   0.6338  -3.1386
10    Topic12   691.263489       

## Calculating similarities

In [36]:
lda_index = similarities.MatrixSimilarity(lda_final[doc_term_matrix])
similarities = lda_index[lda_final[doc_term_matrix]]

def find_similar(doc_id,topn=10):
    similar_docs =sorted(list(enumerate(similarities[doc_id])),key=lambda x: -x[1])[1:]
    return similar_docs[:topn]

In [37]:
find_similar(10,10)

[(10, 1.0),
 (36, 1.0),
 (48, 1.0),
 (62, 1.0),
 (76, 1.0),
 (102, 1.0),
 (117, 1.0),
 (133, 1.0),
 (149, 1.0),
 (166, 1.0)]